# Trabajo Práctico 2: Pruebas de validez

Las pruebas de validez nos permiten probar que un cómputo se ha realizado correctamente en un tiempo bastante menor que la verificación trivial, es decir, que el cómputo sea corrido de manera independiente por el verificador. En este trabajo práctico, nos enfocaremos en el sistema de pruebas STARKs en forma simplificada. Para el trabajo práctico, se podrá hacer uso de las herramientas provistas en https://github.com/starkware-industries/stark101 (python) o https://github.com/lambdaclass/STARK101-rs/tree/main (rust).

## Objetivo

El objetivo consiste en probar la validez de un programa que calcula 2^{8^20} módulo p, utilizando diferentes estrategias y viendo el perfil de costos (tanto en términos de longitud de la prueba como de tiempo de generación). Las estrategias propuestas son:

1. Considerar $a_0 = 2$ y $a_{n+1}=a_n^8$.
2. Considerar $a_0 = 2$ y $a_{n+1}=a_n^2$.
3. Considerar $a_0 = 2$ y $a_{2n+1}=a_{2n}^2$ y $a_{2n}=a_{2n-1}^4$.
4. Usando 2 columnas

Tener en cuenta que, dado el grado de las restricciones, se requiere usar un factor de expansión (blowup) por lo menos tan grande como el grado de las restricciones.

## Estrategia 1: $a_0 = 2$ y $a_{n+1}=a_n^8$

### Polinomio de la traza
Primero vamos a generar la traza. La misma es una lista de 21 elementos, siendo el primero de ellos $a_0 = 2$ y siendo $a_n=a_{n-1}^8$ para $1 \le n \le 20$. De esta manera, el último elemento será $a_{20}=a_{19}^8=2^{8^{20}}$.

In [1]:
from utils.field import FieldElement

a0 = FieldElement(2)
a = [a0]
for i in range(20):
    a.append(a[-1]**8)

In [2]:
#Algunos chequeos de la traza
assert len(a) == 21
assert a[0] == FieldElement(2)
assert a[-1] == FieldElement(2)**(8**20)
print("Traza creada correctamente")

Traza creada correctamente


Ahora hallamos un generador $g$ que nos permita generar un grupo de tamaño 21. 

Sabiendo que $g^k$ genera un grupo de tamaño $\frac {|\mathbb{F}^\times|}{k}$ cuando $k$ divide a $|\mathbb{F}^\times|$, debemos hallar $k$ tal que:

$
21 = \frac {|\mathbb{F}^\times|}{k} \\
21 = \frac{3221225472}{k} \\
k = \frac{3221225472}{21} = 153391689.143
$

Como vemos, $k$ nos queda como un número decimal. Entonces, debemos buscar un número potencia de 2 más próximo a 21 (y más grande) tal que divida a 3221225472, es decir, 32, por eso lo fijamos como el tamaño de nuestro grupo. Ahora sí, calculamos $k$

$
32 = \frac {|\mathbb{F}^\times|}{k} \\
32 = \frac{3221225472}{k} \\
k = \frac{3221225472}{32} = 100663296
$

In [3]:
import math

def get_group(generator, trace_len):
    size = 2 ** math.ceil(math.log2(trace_len))
    k = 3*(2**30)/size
    g = generator ** (k)
    G = [] 
    for i in range(size):
        G.append(g**i)
    
    return g,G

In [4]:
generator = FieldElement.generator()
g,G = get_group(generator,len(a))

In [5]:
#Algunos chequeos del grupo generado
assert g.is_order(32), print("El grupo no tiene orden 32")
assert G[-1]*g == FieldElement(1), print("No es ciclico")
print("Grupo generado correctamente")

Grupo generado correctamente


Por último, para obtener el polinomio de la traza interpolamos los 21 puntos $(x_i,y_i)$ tal que:
- $x_i$: es el elemento $i$ del grupo $G$ generado.
- $y_i$: es el elemento $i$ de la traza

In [6]:
from utils.polynomial import interpolate_poly
f = interpolate_poly(G[:21],a)

### Expansión
Ahora realizamos la expansión, donde evaluamos nuestro polinomio sobre un dominio más grande. Para eso, creamos un grupo nuevo $H$ el cual será 8 veces más grande que $G$ y posteriormente, obtenemos el dominio de evaluación tal que

$$eval\_domain = \{w\cdot h^i | 0 \leq i <256  \}$$

In [7]:
from utils.merkle import MerkleTree
from utils.channel import Channel

def expand_domain(length):
    w = FieldElement.generator()
    h,H =get_group(w, length)
    eval_domain = [w*(h**i) for i in range(len(H))]
    return eval_domain

#Obtenemos dominio de evaluacion
eval_domain = expand_domain(len(G)*8)

#Evaluamos en el polinomio de la traza
f_eval = [f(x) for x in eval_domain]

#Armamos el Merkle tree con las evaluaciones
f_merkle = MerkleTree(f_eval)

#Instanciamos un canal y enviamos la raíz del árbol
channel = Channel()
channel.send(f_merkle.root)

### Restricciones

Tenemos tres restricciones a tener en cuenta:
1. El primer elemento de la traza debe ser igual a $2$. Esto es $a[0] = 2$.
2. El último elemento de la traza debe ser igual a $2^{8^{20}}$. Esto es $a[-1] = 2^{8^{20}} mod(p) = 1610563584$
3. Se debe cumplir que $a[i] = a[i-1]^8$ para todo $1 \leq i \leq 20$

<br>

Podemos expresar estas restricciones en forma polinomial tal que:
1. $f(x)-2$ que tiene como raíz $x=g^0=1$
2. $f(x)-1610563584$ que tiene como raíz $x=g^{20}$
3. $f(g·x) - f(x)^8$ que se anula cuando se evalua en $x$ tal que $x \in G \backslash \{g^{i}|i \geq 20\}$

<br>

Luego, obtenemos las funciones racionales dividiendo las expresiones anteriores por $(x-x')$ tal que $x'$ es raíz de la función del numerador.

#### Restricción 1
$$\frac{f(x)-2}{x-1}$$

#### Restricción 2
$$\frac{f(x)-1610563584}{x-g^{20}}$$

#### Restricción 3
$$\frac{f(g·x)-f(x)^8}{\prod\limits_{i=0}^{19} (x-g^i)}$$

In [8]:
from utils.polynomial import X, prod

#Primera Restricción
numer0 = f - 2
denom0 = X - 1

p0 = numer0 / denom0

#Segunda Restricción
numer1 = f - 1610563584
denom1 = X - g**20

p1 = numer1 / denom1

Para la tercera restricción, aprovechamos la siguiente propiedad:
$$\prod\limits_{i=0}^{31} (x-g^i) = x^{32} - 1$$ 
Veamos que podemos reescribir el denominador de forma que se nos faciliten las cuentas

In [9]:
lst = [X-g**i for i in range(32)]
prod(lst)

In [10]:
#Tercera Restricción
from functools import reduce

numer2 = f(g*X) - f**8
#Cálculo de (X-g^20)*(X-g^21)*...*(X-g^31)
r = reduce(lambda x, y: x * y, [(X-g**i) for i in range(20,32)])
denom2 = ((X**32)-1)/r

p2 = numer2/denom2

### Combinación de polinomios

A partir de una combinación lineal de $p0$,$p1$,$p2$
$$CP(x) = \alpha_0 \cdot p_0(x) + \alpha_1 \cdot p_1(x) + \alpha_2 \cdot  p_2(x)$$

Donde $\alpha_0, \alpha_1, \alpha_2 $ son valores random que los obtenemos dede el verificador. Si nuestra traza cumple con las restricciones, entonces $p0$, $p1$ y $p2$ serán polinomios y la combinación lineal resultante será con alta probabilidad un polinomio también.

Luego, evaluaremos nuestro CP en `eval_domain`.

In [11]:
def get_CP(channel):
    alpha0 = channel.receive_random_field_element()
    alpha1 = channel.receive_random_field_element()
    alpha2 = channel.receive_random_field_element()
    return alpha0*p0 + alpha1*p1 + alpha2*p2

def CP_eval(cp, eval_domain):
    return [cp(d) for d in eval_domain]

cp = get_CP(channel)
cp_eval = CP_eval(cp, eval_domain)

#Armamos el Merkle tree con las evaluaciones
cp_merkle = MerkleTree(cp_eval)

#Enviamos la raíz del árbol
channel.send(cp_merkle.root)

### FRI

Al aplicar FRI, haremos sucesivas iteraciones, donde en cada una reduciremos el dominio de evaluacion y la cantidad de coeficientes a la mitad, reduciendo rambién a la mitad el grado del polinomio. Con esto conseguimos nueva layer.
1. **Nuevo dominio de evaluación**: Nos quedamos con la primer mitad y elevamos al cuadrado los elementos.
2. **Nuevo polinomio**: Separamos en coeficientes de indice par e impar, multiplicamos a los de indice impar por $\beta$ y luego obtenemos los coeficientes del nuevo polinomio realizando la suma: $par + \beta · impar$
3. **Nueva layer**: Evaluamos el nuevo dominio en el nuevo polinomio

In [12]:
from utils.polynomial import Polynomial

#Obtener nuevo dominio
def next_fri_domain(fri_domain):
    return [x ** 2 for x in fri_domain[:len(fri_domain) // 2]]

#Obtener nuevo polinomio
def next_fri_polynomial(poly, beta):
    odd_coefficients = poly.poly[1::2] # No need to fix this line.
    even_coefficients = poly.poly[::2] # No need to fix this line either.
    odd = Polynomial(odd_coefficients)
    even = Polynomial(even_coefficients)
    return even + beta * odd

#Obtener nueva layer
def next_fri_layer(poly, domain, beta):
    next_poly = next_fri_polynomial(poly, beta)
    next_domain = next_fri_domain(domain)
    next_layer = [next_poly(x) for x in next_domain]
    return next_poly, next_domain, next_layer

#Algoritmo FRI + Commitment
def FriCommit(cp, domain, cp_eval, cp_merkle, channel):
    fri_polys = [cp]
    fri_domains = [domain]
    fri_layers = [cp_eval]
    fri_merkles = [cp_merkle]
    
    while fri_polys[-1].degree() > 0:
        beta = channel.receive_random_field_element()
        next_poly, next_domain, next_layer = next_fri_layer(fri_polys[-1], fri_domains[-1], beta)
        fri_polys.append(next_poly)
        fri_domains.append(next_domain)
        fri_layers.append(next_layer)
        fri_merkles.append(MerkleTree(next_layer))
        #Enviamos raiz del Merkle Tree actual
        channel.send(fri_merkles[-1].root)

    #Enviamos constante del último polinomio
    channel.send(str(fri_polys[-1].poly[0]))
    return fri_polys, fri_domains, fri_layers, fri_merkles

fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

## Estrategia 2: $a_0 = 2$ y $a_{n+1}=a_n^2$

### Polinomio de la traza
Primero vamos a generar la traza. La misma es una lista de 81 elementos, siendo el primero de ellos $a_0 = 2$ y siendo $a_n=a_{n-1}^2$ para $1 \le n \le 80$. De esta manera, el último elemento será $a_{80}=a_{79}^2=2^{2^{80}}=2^{8^{20}}$.

In [13]:
a0 = FieldElement(2)
a = [a0]

for i in range(80):
    a.append(a[-1]**2)

#Algunos chequeos de la traza
assert len(a) == 81
assert a[0] == FieldElement(2)
assert a[-1] == FieldElement(2)**(8**20)
print("Traza creada correctamente")

Traza creada correctamente


Similar a la estrategia anterior, necesitamos hallar un generador $g$ que nos permita generar un grupo de tamaño 81. 

Debemos hallar $k$ tal que:

$
81 = \frac {|\mathbb{F}^\times|}{k} \\
81 = \frac{3221225472}{k} \\
k = \frac{3221225472}{81} = 39768215.703
$

Nuevamente, $k$ nos queda como un número decimal. Entonces, debemos buscar un número potencia de 2 más próximo a 81 (y más grande) tal que divida a 3221225472, es decir, 128, por eso lo fijamos como el tamaño de nuestro grupo y lo usamos para calcular $k$

$
128 = \frac{3221225472}{k} \\
k = \frac{3221225472}{128} = 25165824
$

In [14]:
generator = FieldElement.generator()
g,G = get_group(generator,len(a))

In [15]:
#Algunos chequeos del grupo generado
assert g.is_order(128), print("El grupo no tiene orden 128")
assert G[-1]*g == FieldElement(1), print("No es ciclico")
print("Grupo generado correctamente")

Grupo generado correctamente


Obtenemos el polinomio de la traza interpolando los 81 puntos $(x_i,y_i)$ tales que:
- $x_i$: es el elemento $i$ del grupo $G$ generado.
- $y_i$: es el elemento $i$ de la traza

In [16]:
from utils.polynomial import interpolate_poly
f = interpolate_poly(G[:81],a)

### Expansión
El proceso de expansión es exactamente igual que en la estrategia anterior, solo cambia en que G tiene orden 128. Entonces creamos un grupo nuevo $H$ el cual será 8 veces más grande que $G$ y posteriormente, obtenemos el dominio de evaluación tal que

$$eval\_domain = \{w\cdot h^i | 0 \leq i <1024  \}$$

In [17]:
#Obtenemos dominio de evaluacion
eval_domain = expand_domain(len(G)*8)

#Evaluamos en el polinomio de la traza
f_eval = [f(x) for x in eval_domain]

#Armamos el Merkle tree con las evaluaciones
f_merkle = MerkleTree(f_eval)

#Instanciamos un canal y enviamos la raíz del árbol
channel = Channel()
channel.send(f_merkle.root)

### Restricciones

Las primeras dos restricciones son iguales que en la primera estrategia, y la tercera tiene una leve variación:
1. El primer elemento de la traza debe ser igual a $2$. Esto es $a[0] = 2$.
2. El último elemento de la traza debe ser igual a $2^{8^{20}}$. Esto es $a[-1] = 2^{8^{20}} mod(p) = 1610563584$
3. Se debe cumplir que $a[i] = a[i-1]^2$ para todo $1 \leq i \leq 80$

<br>

Podemos expresar estas restricciones en forma polinomial tal que:
1. $f(x)-2$ que tiene como raíz $x=g^0=1$
2. $f(x)-1610563584$ que tiene como raíz $x=g^{80}$
3. $f(g·x) - f(x)^2$ que se anula cuando se evalua en $x$ tal que $x \in G \backslash \{g^{i}|i \geq 80\}$

<br>

Luego, obtenemos las funciones racionales dividiendo las expresiones anteriores por $(x-x')$ tal que $x'$ es raíz de la función del numerador.

#### Restricción 1
$$\frac{f(x)-2}{x-1}$$

#### Restricción 2
$$\frac{f(x)-1610563584}{x-g^{80}}$$

#### Restricción 3
$$\frac{f(g·x)-f(x)^2}{\prod\limits_{i=0}^{79} (x-g^i)}$$

In [18]:
from utils.polynomial import X, prod

#Primera Restricción
numer0 = f - 2
denom0 = X - 1

p0 = numer0 / denom0

#Segunda Restricción
numer1 = f - 1610563584
denom1 = X - g**80

p1 = numer1 / denom1

Similar a la anterior estrategia, para la tercera restricción, reescribimos el denominador aprovechando la propiedad:
$$\prod\limits_{i=0}^{127} (x-g^i) = x^{128} - 1$$ 

In [19]:
#Tercera Restricción
from functools import reduce

numer2 = f(g*X) - f**2
#Cálculo de (X-g^80)*(X-g^81)*...*(X-g^127)
r = reduce(lambda x, y: x * y, [(X-g**i) for i in range(80,128)])
denom2 = ((X**128)-1)/r

p2 = numer2/denom2

### Combinación de polinomios

De la mimsma forma, y utilizando las funciones definidas en la estrategia 1, realizando una combinación lineal de $p0$,$p1$,$p2$ obtenemos
$$CP(x) = \alpha_0 \cdot p_0(x) + \alpha_1 \cdot p_1(x) + \alpha_2 \cdot  p_2(x)$$

Luego, evaluaremos nuestro CP en `eval_domain`.

In [20]:
cp = get_CP(channel)
cp_eval = CP_eval(cp, eval_domain)

#Armamos el Merkle tree con las evaluaciones
cp_merkle = MerkleTree(cp_eval)

#Enviamos la raíz del árbol
channel.send(cp_merkle.root)

### FRI

El proceso de aplicar FRI es exactamente igual al realizado anteriormente en la primera estrategia, por lo que usamos las mismas funciones

In [21]:
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

## Estrategia 3: $a_0 = 2$ y $a_{2n+1}=a_{2n}^2$ y $a_{2n}=a_{2n-1}^4$

### Polinomio de la traza

En primer lugar generamos la traza, donde $a_0 = 2$, y luego los elementos de índice impar se calcularán elevando al cuadrado el elemento anterior, mientras que los de índice par se calcularán elevando a la cuarta el anterior.

In [22]:
a0 = FieldElement(2)
a = [a0]
i = 0
while a[-1] != a0**(8**20):
    i+=1
    a.append(a[-1]**2) if i%2 else a.append(a[-1]**4)

print(f"El tamaño de la traza es: {len(a)}")

El tamaño de la traza es: 20


Ahora hallamos un generador $g$ que nos permita generar un grupo de tamaño 20. Para esto, necesitamos que el tamaño de $G$ sea de $32$ al igual que en la estrategia 1

$
32 = \frac {|\mathbb{F}^\times|}{k} \\
32 = \frac{3221225472}{k} \\
k = \frac{3221225472}{32} = 100663296
$

In [23]:
generator = FieldElement.generator()
g,G = get_group(generator,len(a))

#Algunos chequeos del grupo generado
assert g.is_order(32), print("El grupo no tiene orden 32")
assert G[-1]*g == FieldElement(1), print("No es ciclico")
print("Grupo generado correctamente")

Grupo generado correctamente


Por último, para obtener el polinomio de la traza interpolamos los 20 puntos $(x_i,y_i)$ tal que:
- $x_i$: es el elemento $i$ del grupo $G$ generado.
- $y_i$: es el elemento $i$ de la traza

In [24]:
from utils.polynomial import interpolate_poly
f = interpolate_poly(G[:len(a)],a)

### Expansión

$$eval\_domain = \{w\cdot h^i | 0 \leq i <256  \}$$

In [25]:
#Obtenemos dominio de evaluacion
eval_domain = expand_domain(len(G)*8)

#Evaluamos en el polinomio de la traza
f_eval = [f(x) for x in eval_domain]

#Armamos el Merkle tree con las evaluaciones
f_merkle = MerkleTree(f_eval)

#Instanciamos un canal y enviamos la raíz del árbol
channel = Channel()
channel.send(f_merkle.root)

### Restricciones

Las primeras dos restricciones que fijan el valor del primer y último elemento de la traza se mantienen. Sin embargo, la restricción que determina el valor de los elementos intermedios ahora la vamos a dividir en dos: una para los valores de índice par y otra para los de índice impar.
1. El primer elemento de la traza debe ser igual a $2$. Esto es $a[0] = 2$.
2. El último elemento de la traza debe ser igual a $2^{8^{20}}$. Esto es $a[-1] = 2^{8^{20}} mod(p) = 1610563584$
3. Se debe cumplir que $a[i] = a[i-1]^2$ para todo $i$ tal que $i=1 \ mod(2)$ y $i < 19$
4. Se debe cumplir que $a[i] = a[i-1]^4$ para todo $i$ tal que $i=0 \ mod(2)$ y $i < 19$

<br>

Podemos expresar estas restricciones en forma polinomial tal que:
1. $f(x)-2$ que tiene como raíz $x=g^0=1$
2. $f(x)-1610563584$ que tiene como raíz $x=g^{19}$
3. $f(g·x) - f(x)^2$ que se anula cuando se evalua en $x$ tal que $x \in \{g^i | i=0 \ mod(2), i < 19 \}$
4. $f(g·x) - f(x)^4$ que se anula cuando se evalua en $x$ tal que $x \in \{g^i | i=1 \ mod(2), i < 19 \}$

<br>

Luego, obtenemos las funciones racionales dividiendo las expresiones anteriores por $(x-x')$ tal que $x'$ es raíz de la función del numerador.

#### Restricción 1
$$\frac{f(x)-2}{x-1}$$

#### Restricción 2
$$\frac{f(x)-1610563584}{x-g^{19}}$$

#### Restricción 3
$$\frac{f(g·x)-f(x)^2}{\prod\limits_{n=0}^{9} (x-g^{2n})}$$

#### Restricción 4
$$\frac{f(g·x)-f(x)^4}{\prod\limits_{n=0}^{8} (x-g^{2n+1})}$$

In [26]:
from utils.polynomial import X, prod

#Primera Restricción
numer0 = f - 2
denom0 = X - 1

p0 = numer0 / denom0

#Segunda Restricción
numer1 = f - 1610563584
denom1 = X - g**19

p1 = numer1 / denom1

#Tercera Restricción
numer2 = f(g*X) - f**2
#Cálculo de (X-g^i) tal que i es par
denom2 = reduce(lambda x, y: x * y, [(X-g**i) for i in range(0,19,2)])

p2 = numer2/denom2

#Cuarta Restricción
numer3 = f(g*X) - f**4
#Cálculo de (X-g^i) tal que i es impar
denom3 = reduce(lambda x, y: x * y, [(X-g**i) for i in range(1,19,2)])

p3 = numer3/denom3

### Combinación de polinomios

De la mimsma forma, y utilizando las funciones definidas en la estrategia 1, realizando una combinación lineal de $p0$,$p1$,$p2$,$p3$ obtenemos
$$CP(x) = \alpha_0 \cdot p_0(x) + \alpha_1 \cdot p_1(x) + \alpha_2 \cdot  p_2(x) + \alpha_3 \cdot  p_3(x)$$

Luego, evaluaremos nuestro CP en `eval_domain`.

In [27]:
cp = get_CP(channel)
cp_eval = CP_eval(cp, eval_domain)

#Armamos el Merkle tree con las evaluaciones
cp_merkle = MerkleTree(cp_eval)

#Enviamos la raíz del árbol
channel.send(cp_merkle.root)

## Estrategia 4: Usar dos columnas